In [30]:
import os
import sys
from datetime import datetime

basePath="/Users/david/src/python/StabiHacks/sbbget/sbbget_downloads_orbis/download_temp/"

In [31]:
def printLog(text):
    now = str(datetime.now())
    print("[" + now + "]\t" + text)
    # forces to output the result of the print command immediately, see: http://stackoverflow.com/questions/230751/how-to-flush-output-of-python-print
    sys.stdout.flush()

In [38]:
# check all subdirectories startings with PPN as each PPN stands for a different medium
dirsPerPPN = dict()
ppnDirs=[]
for x in os.listdir(basePath):
    if x.startswith("PPN"):
        dirsPerPPN[x]=[]
        ppnDirs.append(x)

In [39]:
# browse all directories below sbbGetBasePath and search for *_FULLTEXT directories
# and associate each with its PPN
fulltextFilePaths = []

for ppn in ppnDirs:
    for dirpath, dirnames, files in os.walk(basePath+ppn):
        for name in files:
            if dirpath.endswith("_FULLTEXT"):
                # if we found a fulltext directory, only add XML files, i.e., the ALTO candidate files
                if name.endswith(".xml") or name.endswith(".XML"):
                    fulltextFilePaths.append(os.path.join(dirpath, name))
                    dirsPerPPN[ppn].append(os.path.join(dirpath, name))

printLog("Found %i ALTO candidate files for further processing."%len(fulltextFilePaths))
for key in dirsPerPPN:
    print("PPN %s with %i pages."%(key,len(dirsPerPPN[key])))

[2021-03-07 10:06:05.819902]	Found 1731 ALTO candidate files for further processing.
PPN PPN770159389 with 932 pages.
PPN PPN770184375 with 444 pages.
PPN PPN745459102 with 355 pages.


In [36]:
dirsPerPPN

{'PPN770159389': ['/Users/david/src/python/StabiHacks/sbbget/sbbget_downloads_orbis/download_temp/PPN770159389/FILE_0264_FULLTEXT/00000264.xml',
  '/Users/david/src/python/StabiHacks/sbbget/sbbget_downloads_orbis/download_temp/PPN770159389/FILE_0002_FULLTEXT/00000002.xml',
  '/Users/david/src/python/StabiHacks/sbbget/sbbget_downloads_orbis/download_temp/PPN770159389/FILE_0003_FULLTEXT/00000003.xml',
  '/Users/david/src/python/StabiHacks/sbbget/sbbget_downloads_orbis/download_temp/PPN770159389/FILE_0265_FULLTEXT/00000265.xml',
  '/Users/david/src/python/StabiHacks/sbbget/sbbget_downloads_orbis/download_temp/PPN770159389/FILE_0198_FULLTEXT/00000198.xml',
  '/Users/david/src/python/StabiHacks/sbbget/sbbget_downloads_orbis/download_temp/PPN770159389/FILE_0199_FULLTEXT/00000199.xml',
  '/Users/david/src/python/StabiHacks/sbbget/sbbget_downloads_orbis/download_temp/PPN770159389/FILE_0556_FULLTEXT/00000556.xml',
  '/Users/david/src/python/StabiHacks/sbbget/sbbget_downloads_orbis/download_temp

In [1]:
print("test")

test
